# Load point-geometries of European capitals into PostGIS

This is a simplified example.
Coordinates are extracted form GeoJSON, even if there are other ways to load this more efficient into PostGIS.

In [5]:
import json
import os
import psycopg2
import requests

In [7]:
url = 'https://raw.githubusercontent.com/Ginden/capitals/master/europe.json'
r = requests.get(url)

with open('/download/europe.json', 'wb') as f:
    f.write(r.content)

In [8]:
host = 'postgres'
database = 'postgres'
user = 'postgres'
password = os.environ['POSTGRES_PASSWORD']
connection_string = f'host={host} dbname={database} user={user} password={password}'

In [9]:
def execute_simple_sql(sql, fetch='None'):
    with psycopg2.connect(connection_string) as conn:
        with conn.cursor() as curs:
            if fetch=='None':
                curs.execute(sql)
            elif fetch=='One':
                curs.execute(sql)
                result = curs.fetchone();
                return result

In [10]:
sql='CREATE TABLE capitals(gid serial PRIMARY KEY, name TEXT, point geography(POINT,4326) );'
execute_simple_sql(sql)

In [11]:
with open('/download/europe.json') as f:
    data = json.load(f)
    for a_capital in data:
        capital_name = a_capital['properties']['capital']
        lon = a_capital['geometry']['coordinates'][0]
        lat = a_capital['geometry']['coordinates'][1]
        sql = f"INSERT INTO capitals (name, point) VALUES ('{capital_name}', 'SRID=4326;POINT({lon} {lat})');"
        #Please do not use the following function in a loop over a larger dataset
        execute_simple_sql(sql)

## Check the result

Open [PG Admin](http://pgadmin.localhost)

![PgAdminLogin](./images/PgAdminLogin.png)

Enter the username and password that was defined in the credentials.yaml

Right-click on servers and create a new server.


![PgAdminNewServer](./images/PgAdminNewServer.png)

Give it a meaningful name and change to the connection tab.

Host: Postgres

Database: Postgres

User: Postgres

Password: whatever you defined in the credentials.yaml

![PgAdminConnection](./images/PgAdminConnection.png)